In [1]:
from python_prtree import PRTree2D, PRTree3D
import numpy as np
import time
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 18
import random
import gc
np.random.seed(42)
%matplotlib inline

In [2]:
def f(N, PRTree, dim):
    idx = np.arange(N)
    x = np.random.rand(N, 2*dim).astype(np.float32)
    print(x.nbytes // 1024 // 1024) # mb
    for i in range(dim):
        x[:, i+dim] = x[:, i] + x[:, i+dim] / np.sqrt(N) / 100
    s = time.time()
    prtree = PRTree(idx, x)
    t = time.time()
    x = np.random.rand(100_000, 2*dim).astype(np.float32)
    for i in range(dim):
        x[:, i+dim] = x[:, i] + x[:, i+dim] / np.sqrt(N) / 100
    t2 = time.time()
    out = prtree.batch_query(x)
    u = time.time()
    out = [prtree.query(y) for y in x]
    u1 = time.time()
    del_list = random.sample(idx.tolist(), k=min(N, 100_000))
    u2 = time.time()
    for k in del_list:
        prtree.erase(k)
    v = time.time()
    
    x = np.random.rand(min(N, 100_000), 2*dim).astype(np.float32)
    for i in range(dim):
        x[:, i+dim] = x[:, i] + x[:, i+dim] / np.sqrt(N) / 100
    v2 = time.time()
    for i, k in enumerate(del_list):
        prtree.insert(k, x[i])
    return t - s, u - t2, (u1 - u), (v - u2), (time.time() - v2)

In [3]:
for PRTree, name, dim in zip([PRTree2D, PRTree3D], ['2d', '3d'], [2, 3]):
    x = []
    y1 = []
    y2 = []
    y22 = []
    y3 = []
    y4 = []
    for i in list(range(29, 44))[::-1]:
        n = int(1.5 ** i)
        print(n, end=', ')
        x.append(n)
        s, t, t2, u,v = f(n, PRTree, dim)
        y1.append(s)
        y2.append(t)
        y22.append(t2)
        y3.append(u)
        y4.append(v)
        gc.collect()
    plt.figure(figsize=(8, 4))
    plt.plot(x, y1)
    plt.xscale('log')
    plt.xlabel('#rectangles constructed with initial PRTree')
    plt.ylabel('time[sec]')
    plt.title(f'[{name}] construction time')
    plt.savefig(f'./images/{name}_fig1.png')
    plt.clf()
    
    plt.figure(figsize=(8, 4))
    plt.plot(x, y2, label='batch_query')
    plt.plot(x, y22, label='query')
    plt.xscale('log')
    plt.xlabel('#rectangles constructed with initial PRTree')
    plt.legend()
    plt.ylabel('time[sec]')
    plt.title(f'[{name}] total time of 100,000 queries')
    plt.savefig(f'./images/{name}_fig2.png')
    plt.clf()
    
    plt.figure(figsize=(8, 4))
    plt.plot(x, y3, label='erase')
    plt.plot(x, y4, label='insert')
    plt.xscale('log')
    plt.legend()
    plt.xlabel('#rectangles constructed with initial PRTree')
    plt.ylabel('time[sec]')
    plt.title(f'[{name}] total time of 100,000 delete/insert events')
    plt.savefig(f'./images/{name}_fig3.png')
    plt.clf();

37318496, 569
24878997, 379
16585998, 253
11057332, 168
7371554, 112
4914369, 74
3276246, 49
2184164, 33
1456109, 22
970739, 14
647159, 9
431439, 6
287626, 4
191751, 2
127834, 1
37318496, 854
24878997, 569
16585998, 379
11057332, 253
7371554, 168
4914369, 112
3276246, 74
2184164, 49
1456109, 33
970739, 22
647159, 14
431439, 9
287626, 6
191751, 4
127834, 2


<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>

<Figure size 576x288 with 0 Axes>